# `Document & Tutorial`
This BEM package is used for for numerical simulating the electric field of ion traps in python style. Some external packages are wrapped in this package:

* fast triangulation (http://www.cs.cmu.edu/~quake/triangle.research.html)
* fastlap boundary element method (Nabors K S. Efficient three-dimensional capacitance calculation[D]. Massachusetts Institute of Technology, 1993.)

This document, also as a tutorial, explains how to numerical computing electric field and computing its multipole expansion coefficient starting from scratch with a fusion 360 trap model(take `./f3d/htrap_overhang_9x5mm.f3d` for example). Our document consists of three .ipynb files: 

* `I_Mesh_Processing.ipynb`
* `II_Field_Simulation.ipynb`
* `III_Multipole_Expansion.ipynb`

We split document into three files for some reasons:1. the most time consuming part `II_Field_Simulation.ipynb` could be taken out alone and run on HPC. 2.the intermediate result between files can be checked. 

Going through three files in order and running their python code will help you understand our workflow. Generally our workflow divides into three steps, corresponding to the above three .ipynb files. Firstly we identify different electrodes of the trap model by different colors. Also, we refines the trap model meshes to prepare for the second step. The result is stored in `mesh_result.pkl`. Secondly, we simulate the electric potential with fastlap boundary element method(FMM), the accuracy of this step depends on the mesh size in the first step. The result is stored in `field_result.pkl`. Thirdly, we analyze the electric field. Our focus is the multipole coefficient of the field under specific voltage distribution.

Before running codes, you must set up the environment according to instructions in `README`

# `I. Mesh Processing` 
This .pynb file reads a colored model file `./inter_results/htrap.stl`, and assign names for each color(thus for each electrodes). The result is stored in `./inter_results/mesh_result.pkl`.

In this file we name different electrodes and refine the meshes. The size of mesh deternmines the accuracy in next step. You should set the mesh finer and finer until the result converges. Also, you can set the mesh finer and test the convergence.

## (1) color your electrodes (in fusion 360) and export stl
First of all, you have a trap model in fusion 360 project, for example, `./f3d/htrap_overhang_9x5mm.f3d`.A trap always consists of different electrodes. Here we use different color to identify different electrodes(STL file stores color for each mesh). In fusion 360, you can assign different appearance for different parts, of which we only cares about the color(because stl file only stores color), not texture. We recommend to use ../../bemCol_lib/bemCol.adsklib for different electrodes. Same color for two electrodes means they are shorted. Then we get `./f3d/htrap_overhang_9x5mm_my_colored.f3d`

Then export to an stl file in fusion360 file>export tab and move the file in `./inter_results`. We export our stl in mm, so we set

In [1]:
stl_unit = 1e-3

Furthurmore, it's our convention to use unit mm in our reaps. e.g. [Electric field] = V/mm , [harmonic confining potential] = V/mm^2.so we set 

In [2]:
# this is the unit for our Mesh structure
# in the following code, this is the default unit for python variables
mesh_unit = 1e-3    

Some notes:
* here we cut our trap into a smaller one, i.e. `./f3d/htrap_overhang_9x5mm_my_colored_cut.f3d`, before exporting, because distant electrode parts have little influence
* Why use standard color? We define a set of standard color lib ../../../bemCol_lib/bemCol.adsklib for two reasons: 1. when exporting to .stl file, fusion 360 will compress 0-256 color(RGBA32) into 0-32 color(RGBA16), which means that some similar colors in fusion 360 will become same color after export to .stl file. Our standard color lib avoid this problem. 2.If you use .stl file exported from fusion 360 DIRECTLY, naming the electrodes would be very convinient. If you use other apps such as Inventor or Meshlab to generate .stl file, you can ignore the second reason because color encoding are quite different in different apps



## (2) assign a name for each color

In [3]:
import numpy as np
# load stl geometry file
from bem.formats import stl
stl_path = "inter_results/"+"htrap"+".stl"  
s_nta = stl.read_stl(open(stl_path, "rb"))

First of all, we print all the colors need to be named:

In [4]:
sys.path.append('../../bemCol_lib')
from bemCol import bemCol
#from ./bemCol_lib.bemCol import rgb2stl, meshlab_bemCol, bemCol_dict, bemCol
ele_col = bemCol(np.array(list(set(s_nta[2]))),('fusion360','export_stl'))
ele_col.print_colors_to_name()

test_lib {'bem1': (19, 1, 13), 'bem2': (15, 19, 1), 'bem3': (7, 15, 27), 'bem4': (15, 27, 9), 'bem5': (21, 25, 5), 'bem6': (1, 17, 15), 'bem7': (11, 3, 7), 'bem8': (23, 11, 1), 'bem9': (13, 27, 5), 'bem10': (19, 11, 13), 'bem11': (13, 9, 27), 'bem12': (21, 9, 15), 'bem13': (5, 21, 9), 'bem14': (21, 11, 5), 'bem15': (15, 25, 27), 'bem16': (7, 15, 19), 'bem17': (13, 1, 19), 'bem18': (1, 7, 5), 'bem19': (7, 29, 9), 'bem20': (17, 11, 25), 'bem21': (29, 7, 11), 'bem22': (21, 17, 15), 'bem23': (13, 15, 3), 'bem24': (21, 19, 11), 'bem25': (27, 17, 5), 'bem26': (27, 7, 13), 'bem27': (7, 1, 1), 'bem28': (15, 9, 27), 'bem29': (19, 21, 3), 'bem30': (17, 29, 27), 'bem31': (3, 5, 13), 'bem32': (19, 11, 29), 'bem33': (19, 5, 3), 'bem34': (25, 25, 3), 'bem35': (15, 5, 25), 'bem36': (7, 11, 13), 'bem37': (13, 15, 7), 'bem38': (21, 9, 5), 'bem39': (19, 17, 5), 'bem40': (27, 3, 9)}
following are colors you need to name
['bem6']
['bem13']
['bem19']
['bem11']
['bem17']
['bem15']
['bem30']
['bem1']
['bem12

In [5]:
# capture 
mesh = Mesh.from_mesh(stl.stl_to_mesh(*s_nta, scale= mesh_unit/stl_unit, rename=None,print_dropping = False))

NameError: name 'Mesh' is not defined

## (3) remesh

In [ ]:
#!/usr/bin/env python
# coding: utf-8
#here heref!
#more



#   bem: triangulation and fmm/bem electrostatics tools
#
#   Copyright (C) 2011-2012 Robert Jordens <jordens@gmail.com>
#
#   This program is free software: you can redistribute it and/or modify
#   it under the terms of the GNU General Public License as published by
#   the Free Software Foundation, either version 3 of the License, or
#   (at your option) any later version.
#
#   This program is distributed in the hope that it will be useful,
#   but WITHOUT ANY WARRANTY; without even the implied warranty of
#   MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
#   GNU General Public License for more details.
#
#   You should have received a copy of the GNU General Public License
#   along with this program.  If not, see <http://www.gnu.org/licenses/>.

import sys
from time import time
import numpy as np
import resource
import matplotlib as mpl
import matplotlib.pyplot as plt
import multiprocessing 
multiprocessing.set_start_method("fork")


import numpy as np

sys.path.append('../../')
from bem import Electrodes, Sphere, Mesh, Grid, Configuration, Result
from bem.formats import stl
import numpy as np
import copy
from helper_functions import *
from bemCol_lib.bemCol import rgb2stl, meshlab_bemCol, bemCol_dict, bemCol




# ### Import STL geometry file
# base file name for outputs and inputs is the script name

stl_file_in = "htrapf"
# scale to natural units (ion height)
# this seems not right to me- I feel like the ion-to-electrode distance is own for a spherical
# electrode geometry
leng = 1
factor = 1
# 72/leng
scale = 1e-3   # Distance from ion to electrode is 40 um.
use_stl = True

# we use a finite difference method to test converge:
# we set the mesh double finer, if the change of field is small, then we regard it converge
test_cvg = False


mesh,s_nta = load_file(Mesh,Electrodes,stl_file_in,scale,use_stl)
# The formal rename of electrode. Assign each electrode a string name instead of its color coding. Use the numbers you get above.
# `stl.stl_to_mesh()` prints normal vectors (different faces) in each electrode.

# print color with rename: 'bem1','bem2','uk1','uk2', etc
# s_nta[2] returns a set of attributes
print('test num color:',np.array(list(set(s_nta[2]))))
ele_col = bemCol(np.array(list(set(s_nta[2]))),('fusion360','export_stl'))
ele_col.print_colors_to_name()

ele_col.set_color_name(color = 'bem1',name = 'DC1')
ele_col.set_color_name(color = 'bem2',name = 'DC2')
ele_col.set_color_name(color = 'bem3',name = 'DC3')
ele_col.set_color_name(color = 'bem4',name = 'DC4')
ele_col.set_color_name(color = 'bem5',name = 'DC5')
ele_col.set_color_name(color = 'bem6',name = 'DC6')
ele_col.set_color_name(color = 'bem7',name = 'DC7')
ele_col.set_color_name(color = 'bem8',name = 'DC8')
ele_col.set_color_name(color = 'bem9',name = 'DC9')
ele_col.set_color_name(color = 'bem10',name = 'DC10')
ele_col.set_color_name(color = 'bem11',name = 'DC11')
ele_col.set_color_name(color = 'bem12',name = 'DC12')
ele_col.set_color_name(color = 'bem13',name = 'DC13')
ele_col.set_color_name(color = 'bem14',name = 'DC14')
ele_col.set_color_name(color = 'bem15',name = 'DC15')
ele_col.set_color_name(color = 'bem16',name = 'DC16')
ele_col.set_color_name(color = 'bem17',name = 'DC17')
ele_col.set_color_name(color = 'bem18',name = 'DC18')
ele_col.set_color_name(color = 'bem19',name = 'DC19')
ele_col.set_color_name(color = 'bem20',name = 'DC20')
ele_col.set_color_name(color = 'bem21',name = 'DC21')
ele_col.set_color_name(color = 'bem30',name = 'DC0')
ele_col.set_color_name(color = 'bem25',name = 'RF')

ele_col.print_drop_colors()

print(len(s_nta), type(s_nta),"\n")
# s_nta is a length 3 tuple. (normal, triangle, attribute)
# Normal direction of each triangle, three vetice of triangles, coding number of colors.
print("Triangles:",len(s_nta[0]),"\nColors:",len(s_nta[2]),"\n")    # This isn't right.

# stl_to_mesh() only assigns names and does scaling, doing no triangulation to stl mesh.
# "scale=scale/1e-6" only scales dimensionless scale/1e-6.    1e-6: if stl uses micron as unit.
color_dict = ele_col.result_dict
print('test_rename',color_dict)
mesh = Mesh.from_mesh(stl.stl_to_mesh(*s_nta, scale=1,
    rename=color_dict, quiet=False))



# ### Generate triangle mesh with constraints
#
# The meshes are 2-dimensional triangles on the surface of electrodes. The region enclosed by constraint shape can have finer mesh. Triangulation is done by `triangle` C library.
#there are all in units of mm now (Ben S. feb 2022)
xl = (3.1)*72*1e-3
yl = -0.051*72*1e-3
zl = 1.06*72*1e-3
rad = 5*72*1e-3
size = 100.0
inside=2e-4
outside=2e-3
# set .1 max area within 3
# areas_from_constraints specifies sphere with finer mesh inside it.
mpl.rcParams['lines.linewidth'] = 0.2 ###########

plot_mesh(xl,yl,mesh,scale,'fig1.png')

mesh.triangulate(opts="q10Q",new = False)
plot_mesh(xl,yl,mesh,scale,'fig2.png')
 # "inside", "outside" set different mesh densities.Q
# mesh.areas_from_constraints(Sphere(center=np.array([xl,yl,zl]),
#            radius=10*factor, inside=0.1*factor**2, outside=1000))
# mesh.triangulate(opts="",new = False)
print('--------------')
mesh.areas_from_constraints(Sphere(center=np.array([xl,yl,zl]),
           radius=rad, inside=inside, outside=outside))
# # retriangulate quality and quiet with areas
mesh.triangulate(opts="q20Q",new = False)
plot_mesh(xl,yl,mesh,scale,'fig3.png')
if test_cvg:
    mesh_fine = copy.deepcopy(mesh)
    mesh_fine.areas_from_constraints(Sphere(center=np.array([xl,yl,zl]),radius=rad, inside=inside/2, outside=outside/2))
    mesh_fine.triangulate(opts="q25Q",new = False)


# save base mesh to vtks
#print("Output vtk:",os.path.abspath("./"+stl_file_in+".vtk"))    # output path

# save base mesh to a pickle file
fout = 'htrap_simulation_1_el3.5'
with open('./'+fout+'_meshResult'+'.pkl','wb') as f:
    pickle.dump(mesh,f)
